In [44]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
torch.manual_seed(42)

In [ ]:
# input from txt
file=open("data.txt","r")
x_train=np.empty(1)
y_train=np.empty(1)

for i in file.readlines():
    if " " in i:
        valori = i.split()
        valori = list(map(lambda x: float(x),valori))
        y_train=np.append(y_train, valori[0])
        x_train=np.append(x_train, valori[1:])
x_train = x_train[1:]
y_train = y_train[1:]
x_train = x_train.reshape(-1,6)
y_train = y_train.reshape(-1,1)
# y_train = np.log(y_train)

file2=open("data2.txt","r")
x_test=np.empty(1)
y_test=np.empty(1)

for i in file2.readlines():
    if " " in i:
        valori = i.split()
        valori = list(map(lambda x: float(x),valori))
        y_test=np.append(y_test, valori[0])
        x_test=np.append(x_test, valori[1:])
x_test = x_test[1:]
y_test = y_test[1:]
x_test = x_test.reshape(-1,6)
y_test = y_test.reshape(-1,1)
# y_test = np.log(y_test)

In [ ]:
X_train = torch.from_numpy(x_train).float()
y_train = torch.squeeze(torch.from_numpy(y_train).float())
X_test = torch.from_numpy(x_test).float()
y_test = torch.squeeze(torch.from_numpy(y_test).float())

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 7)
        self.fc2 = nn.Linear(7, 4)
        self.fc3 = nn.Linear(4, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))
net = Net(X_train.shape[1])

In [ ]:
criterion = nn.PoissonNLLLoss
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [ ]:
# use this if you have GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

net = net.to(device)
criterion = criterion.to(device)

In [ ]:
def round_tensor(t, decimal_places=3):
    return round(t.item(), decimal_places)
for epoch in range(10000):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 500 == 0:
        y_test_pred = net(X_test)
        y_test_pred = torch.squeeze(y_test_pred)
        test_loss = criterion(y_test_pred, y_test)
        print('epoch ',epoch,'Train set - loss: ',round_tensor(train_loss),'Test  set - loss: ',round_tensor(test_loss))
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

In [ ]:
# save model
MODEL_PATH = 'model.pth'
torch.save(net, MODEL_PATH)

In [ ]:
#load model
net = torch.load(MODEL_PATH)

In [ ]:
def predict(size):
    t = torch.as_tensor(size).float().to(device)
    output = net(t)
    return output

In [ ]:
predict([1000,444,0,0,0,0])